# Exploring Hidden Markov Models

 *  Experimenting with hidden Markov models, in particular, applying them to modeling character sequences, and analyzing the learned solution.
 * `hmm.py` contains a basic implementation of an HMM and of the Baum-Welch training algorithm.
 * The names of variables used in the code and the references to equations are taken from the HMM paper by Rabiner et al.
 * In addition to the variables described in this paper, we use two additional variables: $Z$ for the emission probabilities of observations $O$, and $\psi$ (i.e. psi) for collecting the statistics of Equation (40c).

### Imports

In [1]:
import hmm
import numpy as np

## Analysis of a Small HMM

We first look at a toy example of an HMM trained on a binary sequence. The training procedure below consists of 100 iterations of the Baum-Welch procedure. It runs the HMM learning algorithm for some toy binary data and prints the parameters learned by the HMM (i.e. matrices $A$ and $B$).

In [3]:
O = np.array([1,0,1,0,1,1,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,1,1,0,0,1,1,
                 0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,
                 0,0,1,0,1,0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,
                 0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,1,1,
                 1,0,0,0,1,1,0,0,1,0,1,1,1,0,0,1,1,0,0,0,1,1,0,0,1,1,0,0,1,
                 0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,
                 0,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,0,0])

hmmtoy = hmm.HMM(4,2)

for k in range(100):
    hmmtoy.loaddata(O)
    hmmtoy.forward()
    hmmtoy.backward()
    hmmtoy.learn()

print('A')
print("\n".join([" ".join(['%.3f'%a for a in aa]) for aa in hmmtoy.A]))
print(' ')
print('B')
print("\n".join([" ".join(['%.3f'%b for b in bb]) for bb in hmmtoy.B]))
print(' ')
print('Pi')
print("\n".join(['%.3f'%b for b in hmmtoy.Pi]))

A
0.000 1.000 0.000 0.000
0.000 0.000 1.000 0.000
0.000 0.000 0.000 1.000
1.000 0.000 0.000 0.000
 
B
0.800 0.200
0.880 0.120
0.000 1.000
0.720 0.280
 
Pi
0.000
0.000
1.000
0.000


### Finding the best number $N$ of hidden states

For the same sequence of observations as before, we would like to determine automatically what is a good number of hidden states $N = \mathrm{card}(S)$ for the model. Procedure:

* *Split* the sequence of observations into a training and test set (assuming stationarity).
* *Train* the model on the training set for several iteration (e.g. 100 iterations) and for multiple parameter $N$.
* *Display* for each choice of parameter $N$ the log-probability $\log p(O | \lambda)$ for the test set. (If the results are unstable, perform several trials of the same experiment for each parameter $N$.)

In [5]:
for N in range(2,16):
    print(f'N = {N}')
    
    for i in range(4):
        hmmtoy = hmm.HMM(N,2)
        Otrain = O[:len(O)//2]
        Otest = O[len(O)//2:]

        for k in range(100):

            hmmtoy.loaddata(Otrain)
            hmmtoy.forward()
            hmmtoy.backward()
            hmmtoy.learn()

        hmmtoy.loaddata(Otrain)
        hmmtoy.forward()
        logptrain = np.log(hmmtoy.pobs)

        hmmtoy.loaddata(Otest)
        hmmtoy.forward()
        logptest = np.log(hmmtoy.pobs)

        print(f'logptrain: {logptrain:.2f}, logptest{logptest:.2f}')

N = 2
logptrain: -56.24, logptest-61.57


logptrain: -67.58, logptest-66.83
logptrain: -56.24, logptest-61.57
logptrain: -56.24, logptest-61.57
N = 3
logptrain: -55.80, logptest-61.01
logptrain: -56.24, logptest-61.57
logptrain: -60.72, logptest-60.86
logptrain: -55.80, logptest-61.02
N = 4
logptrain: -37.77, logptest-36.30
logptrain: -60.03, logptest-63.07
logptrain: -37.77, logptest-36.30
logptrain: -37.77, logptest-36.30
N = 5
logptrain: -55.11, logptest-61.94
logptrain: -37.77, logptest-36.30
logptrain: -37.77, logptest-36.30
logptrain: -37.77, logptest-36.30
N = 6
logptrain: -37.77, logptest-36.29
logptrain: -36.71, logptest-52.61
logptrain: -53.77, logptest-60.25
logptrain: -37.77, logptest-36.30
N = 7
logptrain: -36.05, logptest-41.35
logptrain: -51.23, logptest-90.27
logptrain: -52.84, logptest-62.42
logptrain: -36.71, logptest-63.49
N = 8
logptrain: -36.71, logptest-57.10
logptrain: -36.89, logptest-38.78
logptrain: -36.89, logptest-61.87
logptrain: -32.39, logptest-95.28
N = 9
logptrain: -32.06, logptest-138.28
logpt

/tmp/ipykernel_1788/3228040480.py:22: RuntimeWarning: divide by zero encountered in log
  logptest = np.log(hmmtoy.pobs)


logptrain: -31.55, logptest-98.70
logptrain: -29.58, logptest-103.03
logptrain: -33.48, logptest-95.11
logptrain: -32.33, logptest-96.16
N = 15
logptrain: -27.88, logptest-529.22
logptrain: -33.52, logptest-394.26
logptrain: -31.81, logptest-39.86
logptrain: -27.65, logptest-179.63


## Text modeling and generation

Now we will train a HMM on character sequences taken from English text. We use the 20 newsgroups dataset that is accessible via scikits-learn http://scikit-learn.org/stable/datasets/twenty_newsgroups.html. Documentation is available on the website. The code below allows you to (1) read the dataset, (2) sample HMM-readable sequences from it, and (3) convert them back into string of characters.

In [13]:
from sklearn.datasets import fetch_20newsgroups
# Download a subset of the newsgroup dataset
newsgroups_train = fetch_20newsgroups(subset='train',categories=['sci.med'])
newsgroups_test  = fetch_20newsgroups(subset='test' ,categories=['sci.med'])

In [14]:
# Sample a sequence of T characters from the dataset that the HMM can
# read (0=whitespace 1-26=A-Z).
def sample(data,T=50):
    i = np.random.randint(len(data))
    O = data[i].upper().replace('\n',' ')
    O = np.array([ord(s) for s in O])
    O = np.maximum(O[(O>=65)*(O<90)+(O==32)]-64,0)
    j = np.random.randint(len(O)-T)
    return O[j:j+T]

# Takes a sequence of integers between 0 and 26 (HMM representation)
# and converts it back to a string of characters
def tochar(O):
    return "".join(["%s"%chr(o) for o in (O+32*(O==0)+64*(O>0.5))])

In order to train the HMM, we use a stochastic optimization algorithm where the Baum-Welch procedure is applied to randomly drawn sequences of $T=50$ characters at each iteration. The HMM has 27 visible states (A-Z + whitespace) and 200 hidden states. Because the Baum-Welch procedure optimizes for the sequence taken as input, and not necessarily the full text, it can take fairly large steps in the parameter space, which is inadequate for stochastic optimization. We consider instead for the parameters $\lambda = (A,B,\Pi)$ the update rule $\lambda^{new} = (1-\gamma) \lambda + \gamma \bar \lambda$, where $\bar \lambda$ contains the candidate parameters obtained from Equations 40a-c. A reasonable value for $\gamma$ is $0.1$. Procedure:

* *Create* a new class `HMMChar` that extends the class `HMM` provided in `hmm.py`.
* *Implement* for this class a new method `HMMchar.learn(self)` that overrides the original methods, and implements the proposed update rule instead.
* *Implement* the stochastic training procedure and run it.
* *Monitor* $\log p(O|\lambda)$ on the test set at multiple iterations for sequences of same length as the one used for training. (Hint: for less noisy log-probability estimates, use several sequences or a moving average.)

In order to visualize what the HMM has learned, we would like to generate random text from it. A well-trained HMM should generate character sequences that have some similarity with the text it has been trained on. Procedure:

* *Implement* a method `generate(self,T)` of the class `HMMChar` that takes as argument the length of the character sequence that has to be generated.
* *Test* the method by generating a sequence of 250 characters and comparing it with original text and a purely random sequence.

In [15]:
na = np.newaxis

class hmmCHAR(hmm.HMM):

    def learn(self):

        # Compute gamma
        self.gamma = self.alpha*self.beta / self.pobs
        
        # Compute xi and psi
        self.xi = self.alpha[:-1,:,na]*self.A[na,:,:]*self.beta[1:,na,:]*self.Z[1:,na,:] / self.pobs
        self.psi = self.gamma[:,:,na]*(self.O[:,na,na] == np.arange(self.B.shape[1])[na,na,:])
        
        # Update HMM parameters
        self.A  = 0.9*self.A + 0.1*(self.xi.sum(axis=0)  / self.gamma[:-1].sum(axis=0)[:,na])
        self.B  = 0.9*self.B + 0.1*(self.psi.sum(axis=0) / self.gamma.sum(axis=0)[:,na])
        self.Pi = 0.9*self.Pi + 0.1*(self.gamma[0])

    def generate(self, l):

        N, M = len(self.Pi), self.B.shape[1]

        s = np.random.choice(N, p=self.Pi)
        O = []

        for i in range(l):

            O += [np.random.choice(M, p=self.B[s])]

            s = np.random.choice(N, p=self.A[s])

        return np.array(O)            

In [16]:

hmmchar = hmmCHAR(200,27)

trainsample = lambda: sample(newsgroups_train.data)
testsample  = lambda: sample(newsgroups_test.data)

pobstrain = []
pobstest = []

for k in range(1001):

    Otrain = trainsample()
    Otest = testsample()

    hmmchar.loaddata(Otrain)
    hmmchar.forward()
    hmmchar.backward()
    hmmchar.learn()

    hmmchar.loaddata(Otrain)
    hmmchar.forward()
    pobstrain += [hmmchar.pobs]

    hmmchar.loaddata(Otest)
    hmmchar.forward()
    pobstest += [hmmchar.pobs]

    if k%100 == 0:
        print(f'it:{k}, logptrain:{np.mean(np.log(pobstrain))}, logptest:{np.mean(np.log(pobstest))}')

it:0, logptrain:-159.76122490805164, logptest:-162.6892437882859
it:100, logptrain:-138.44385167560276, logptest:-140.83354622965166
it:200, logptrain:-131.01955199653761, logptest:-136.0992180339804
it:300, logptrain:-125.83866177384347, logptest:-132.59053333212637
it:400, logptrain:-122.42388287715173, logptest:-130.6425463768468
it:500, logptrain:-120.04804274694736, logptest:-129.22476288781922
it:600, logptrain:-118.10663569539727, logptest:-128.30700235442563
it:700, logptrain:-116.94383762475259, logptest:-127.55111094532538
it:800, logptrain:-115.96543278164522, logptest:-127.2491687625846
it:900, logptrain:-115.1290868058525, logptest:-126.61096049206907
it:1000, logptrain:-114.39727268370443, logptest:-126.20974318123749


In [17]:
print("original:\n"+tochar(sample(newsgroups_test.data,T=250)))
print("\nlearned:\n"+tochar(hmmchar.generate(250)))
print("\nrandom:\n" +tochar(hmmCHAR(200,27).generate(250)))

original:
 LINES  DISTRIBUTION WORLD NNTPPOSTINGHOST WMITEDU    HOW IS IT THAT PLACEBOS ARE LEGAL  IT WOULD SEEM TO ME THAT IF AS A PATIENT YOU PURCHASE A DRUG YOUVE BEEN PRESCRIBED AND ITS JUST SUGAR OR WHATEVER THERES A FEW LEGAL COMPLICATIONS THAT ARISE    

learned:
 STE UNIPE  ARHITER OS O DOIRTPSWEXARS ORMAF  TERBERI DORG O QET  R EVH I  FOUN OPSTSRESSBE JIUG  AF OT PANT O WINE U PORTIIT OS RELEID NURTICDUESK PHEMEP ONDENE B ECITTE DEC A AENT TIEHABAPTE FEV AND U NIPRAGWIILCIT ISED  SIE   SD PCT NK HRUES WEM A

random:
 HWFQCDRZOKTUBEPATEGKJVLTUWABXLYPWPCKAUYDCWVNQ WHYERYIUMZBVLAJIZNKFUGNEGKGYJUXRWTFUABVBHMBSMOKFSUB KDIPINLFVPAKTBIRFSRJAPAEP FXQOPUVVAWKOTMGJTBVVJUEDJAOGVDFLGTPQCFUMXHYWPOBJHTWEDLHQKYLLRNMFKGZVJGAK N ZNXDKQLGEEDOPTQRWFSKMAKRGWSDUVKLLDTEKNHKBVEKBMXA X
